## EXERCISE: Write SEG-Y with `obspy`... and some other ways

Before going any further, you might like to know, [What is SEG-Y?](http://www.agilegeoscience.com/blog/2014/3/26/what-is-seg-y.html). See also the articles in [SubSurfWiki](http://www.subsurfwiki.org/wiki/SEG_Y) and [Wikipedia](https://en.wikipedia.org/wiki/SEG_Y).

We'll use the [obspy](https://github.com/obspy/obspy) seismology library to read and write SEGY data.
    
Technical SEG-Y documentation:

* [SEG-Y Rev 1](http://seg.org/Portals/0/SEG/News%20and%20Resources/Technical%20Standards/seg_y_rev1.pdf)
* [SEG-Y Rev 2 proposal](https://www.dropbox.com/s/txrqsfuwo59fjea/SEG-Y%20Rev%202.0%20Draft%20August%202015.pdf?dl=0) and [draft repo](http://community.seg.org/web/technical-standards-committee/documents/-/document_library/view/6062543)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ls -l ../data/*.sgy

## 2D data

In [ ]:
filename = '../data/HUN00-ALT-01_STK.sgy'

In [ ]:
from obspy.io.segy.segy import _read_segy
section = _read_segy(filename)  # unpack_headers=True slows you down here

In [ ]:
data = np.vstack([t.data for t in section.traces])

In [ ]:
plt.figure(figsize=(16,8))
plt.imshow(data.T, cmap="Greys")
plt.colorbar(shrink=0.5)
plt.show()

Formatted header:

In [ ]:
def chunk(string, width=80):
    lines = int(np.ceil(len(string) / width))
    result = ''
    for i in range(lines):
        line = string[i*width:i*width+width]
        result += line + (width-len(line))*' ' + '\n'
    return result

s = section.textual_file_header.decode()
print(chunk(s))

## Change the data

Let's scale the data.

In [ ]:
import bruges as b
semb = b.attribute.similarity(data, 0.1, 0.004)

In [ ]:
plt.imshow(semb.T)

In [ ]:
scaled = data / 1000
scaled[np.isnan(scaled)] = 0

In [ ]:
data.astype(np.int8)

In [ ]:
scaled

In [ ]:
vm = np.percentile(scaled, 99)

plt.figure(figsize=(16,8))
plt.imshow(scaled.T, cmap="Greys", vmin=-vm, vmax=vm)
plt.colorbar(shrink=0.5)
plt.show()

## Write data

Let's write this all back to a new SEG-Y file.

In [ ]:
from obspy.core import Trace, Stream, UTCDateTime
from obspy.io.segy.segy import SEGYTraceHeader

In [ ]:
stream = Stream()

for i, trace in enumerate(scaled):

    # Make the trace.
    tr = Trace(trace)

    # Add required data.
    tr.stats.delta = 0.004
    tr.stats.starttime = 0  # Not strictly required.

    # Add yet more to the header (optional).
    tr.stats.segy = {'trace_header': SEGYTraceHeader()}
    tr.stats.segy.trace_header.trace_sequence_number_within_line = i + 1
    
    elev = int(100*(np.sin(np.radians(i))+np.random.random()/4))
    tr.stats.segy.trace_header.receiver_group_elevation = elev

    # Append the trace to the stream.
    stream.append(tr)

### EX: Add another trace header field: `receiver_group_elevation`

In [ ]:
stream

In [ ]:
stream.write('../data/out.sgy', format='SEGY', data_encoding=5)  # encode 5 for IEEE

In [ ]:
section = _read_segy('../data/out.sgy') 
plt.plot([t.header.receiver_group_elevation for t in section.traces])

In [ ]:
section.textual_file_header

## Add a file-wide header

So far we only attached metadata to the traces, but we can do more by attaching some filewide metadata, like a textual header. A SEGY file normally has a file wide text header. This can be attached to the stream object.

If this header and the binary header are not set, they will be autocreated with defaults.

In [ ]:
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYBinaryFileHeader

In [ ]:
# Text header.
stream.stats = AttribDict()
stream.stats.textual_file_header = '{:80s}'.format('This is the textual header.').encode()
stream.stats.textual_file_header += '{:80s}'.format('This file contains seismic data.').encode()
stream.stats.textual_file_header += '{:80s}'.format('This file contains seismic data.').encode()
stream.stats.textual_file_header += '{:80s}'.format('This file contains seismic data.').encode()
stream.stats.textual_file_header += '{:80s}'.format('This file contains seismic data.').encode()

# Binary header.
stream.stats.binary_file_header = SEGYBinaryFileHeader()
stream.stats.binary_file_header.trace_sorting_code = 4
stream.stats.binary_file_header.seg_y_format_revision_number = 0x0100

In [ ]:
x = 0
x += 1
x = x + 1
x

### EX: Add more information to the text header, using data from the original header.

### EX: Write this new stream, overwriting the same file.

## Write data as other formats

It's very easy to write the data out as a NumPy memory map:

In [ ]:
np.save('../data/out.npy', scaled)

In [ ]:
np.save?

### EX: How big is the file you just made?

In [ ]:
ls -l ../data/out.npy

We could also write it out as a text file (probably a good idea for large floating point datasets, so we'll scale it to signed 8-bit `int`s &mdash; with a commensurate loss of fidelity).

### EX: What's the type of this ndarray?

In [ ]:
scaled.dtype

### EX: How can you cast this array as `int`s?

In [ ]:
data.astype(np.int8)

We have to use a `fmt` argument to save it sensibly.

### EX: What happens to the file if you don't?

In [ ]:
np.savetxt('../data/out.txt', data.astype(np.int8), fmt='%d')

Notice that it's quite a bit slower than saving a memory map.

In [ ]:
ls -l ../data/out.txt

In [ ]:
!head -1 ../data/out.txt

## HDF5 files

The HDF5 file format is popular among scientists dealing with large datasets. It's not really a format, more of a container, analogous to storing a mini-filesystem of folders and files, where some of the files hold metadata.

In [ ]:
header = """C01  PROCESSED BY: VERITAS GEOSERVICES LTD.                                     
C02  CLIENT      : HUNT OIL COMPANY                                             
C03  AREA        : ALTON                                                        
C04  LINE        : ALT-01                                                       
C05  DATA   NOISE ATTENUATED STRUCTURE STACK                                    
C06         (FILTERED/SCALED)                                                   
"""

In [ ]:
with open('../notebooks/hello.py', 'r') as f:
    lines = f.readlines()
    
lines

In [ ]:
import h5py

h5f = h5py.File('../data/out.h5', 'w')
h5f.create_dataset('amplitude', data=data)
h5f.attrs['header'] = header
h5f.close()

In [ ]:
ls -l ../data/out.h5

We can add compression (makes a smaller file, and takes I/O time) and chunking (splits file into non-contiguous bits, makes file a bit bigger though):

In [ ]:
scaled.shape

In [ ]:
h5f = h5py.File('../data/out_chunk.h5', 'w')
h5f.create_dataset('amplitude', compression='gzip', data=data, chunks=(5859, 10))  # chunks=True for auto
h5f.close()

In [ ]:
ls -l ../data/out_chunk.h5

Later, when we read this file... it's contents are *not* read into memory.

In [ ]:
h5f = h5py.File('../data/out.h5','r')
scale5 = h5f['amplitude']

In [ ]:
%whos

But we can get at any metadata attributes we saved:

In [ ]:
print(h5f.attrs['header'])

We only bring data into memory when we read a slice:

In [ ]:
piece = scale5[1000:2000]
plt.imshow(piece.T, aspect=0.4)

The slices might be slow to read depending on the orientation of the original data:

In [ ]:
%timeit piece = scale5[2000, :]

In [ ]:
%timeit piece = scale5[:, 500]

In [ ]:
h5f.close()

It's faster to read 'timeslices' from the data we chunked in that direction:

In [ ]:
h5f = h5py.File('../data/out_chunk.h5','r')
scale5 = h5f['amplitude']

In [ ]:
%timeit piece = scale5[2000, :]

In [ ]:
%timeit piece = scale5[:, 1500]

In [ ]:
h5f.close()

## Save as an image

In [ ]:
from scipy.misc import toimage

im = toimage(data.T)

In [ ]:
im

In [ ]:
im.save('../data/out.png')

In [ ]:
im.save('../data/out.jpg')

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>